# Installing Model Visualizer

In [ ]:
!pip install visualkeras

# Fetch Necessary Libraries

In [ ]:
import plotly.offline as pyo
pyo.init_notebook_mode()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn import preprocessing
import random
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

# Loading data from file

In [ ]:
def Create_Directory_DataFrame():
    df =pd.DataFrame(columns=['Class','Location'])
    basedir = '../input/leapgestrecog/leapGestRecog/'
    for folder in os.listdir(basedir):
        for Class in os.listdir(basedir+folder+'/'):
            for location in os.listdir(basedir+folder+'/'+Class+'/'):
                df = df.append({'Class':Class,'Location':basedir+folder+'/'+Class+'/'+location},ignore_index=True)
    df = df.sample(frac = 1) 
    return df

In [ ]:
df = Create_Directory_DataFrame()
print(df.shape)
df.head()

# Visualize Data

In [ ]:
count = 1
f = plt.figure(figsize=(50,13))
for Class in df['Class'].unique():
    seg = df[df['Class']==Class]
    address =  seg.sample().iloc[0]['Location']
    img = cv2.imread(address,0)
    ax = f.add_subplot(2, 5,count)
    ax = plt.imshow(img)
    ax = plt.title(Class,fontsize= 30)
    count = count + 1
plt.suptitle("Hand Sign Images", size = 32)
plt.show()

In [ ]:
img.shape

# Define Constraints

In [ ]:
w , h= 64,64
final_class = 10

# Data Creation

In [ ]:
from tqdm import tqdm
train_image = []
for location in tqdm(df.iloc[:]['Location']):
    img = cv2.imread(location,0)
    img = cv2.resize(img, (w,h), interpolation = cv2.INTER_AREA)
    img = img.reshape(w,h,1)
    train_image.append(img)
X = np.array(train_image)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
y = np.array(df.iloc[:]['Class'])
y = y.reshape(y.shape[0],1)
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y)
print(enc.categories_)

In [ ]:
y = enc.transform(y).toarray()

In [ ]:
print('Data   :   '+str(X.shape))
print('Output :   '+str(y.shape))

# Samples

In [ ]:
print(X[0].reshape(w,h))

In [ ]:
plt.figure(figsize=(25,8))
plt.imshow(X[66].reshape(w,h))
plt.title(enc.inverse_transform(y[0].reshape(1,10))[0][0],size = 20)
plt.show()

In [ ]:
y[0]

# Segmentation in Traing and Test Data Sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
print('Train data    :'+str(X_train.shape))
print('Test data     :'+str(X_test.shape))
print('Train Output  :'+str(y_train.shape))
print('Test Output   :'+str(y_test.shape))

# Model

1. To make our model more modular and easier to understand, let's define some blocks. As we're building a convolution neural network, we'll create a convolution block and a dense layer block.
2. The following method will define the function to build our model for us. The Dropout layers are important as they "drop out," hence the name, certain nodes to reduce the likelikhood of the model overfitting. We want to end the model with a Dense layer of one node, as this will be the output that determines if an X-ray shows an image of pneumonia.
Since there are only two possible labels for the image, we will be using the binary_crossentropy loss. When we fit the model, identify the class weights. Because we are using a TPU, training will be relatively quick.

For our metrics, we want to include precision and recall as they will provide use with a more informed picture of how good our model is. Accuracy tells us what fractions are the labels are correct. Since our data is not balanced, accuracy might give a skewed sense of a good model (i.e. a model that always predicts PNEUMONIA will be 74% accurate but is not a good model).

Precision is the number of true positives (TP) over the sum of TP and false positives (FP). It shows what fraction of labeled positives are actually correct.

Recall is the number of TP over the sum of TP and false negatves (FN). It shows what fraction of actual positives are correct.

In [ ]:
def conv_block(filters):
    block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ]
    )
    return block
def dense_block(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    return block
def build_model(act , final_class , w , h ):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(w , h , 1)),
        
        tf.keras.layers.Conv2D(16, 3, activation=act, padding='same'),
        tf.keras.layers.Conv2D(16, 3, activation=act, padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        conv_block(32),
        conv_block(64),
        
        conv_block(128),
        tf.keras.layers.Dropout(0.2),
        
        conv_block(256),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
        
        tf.keras.layers.Dense(final_class, activation='sigmoid')
    ])
    return model

In [ ]:
def wrap(Training_Output_Results , Opt , Act ,  history):
    epoch  = len(history.history['loss'])
    epochs = list(np.arange(1,epoch + 1,1))
    Optimizer = np.repeat(Opt,epoch).tolist()
    Activation = np.repeat(Act,epoch).tolist()
    cumiliated_res = {}
    cumiliated_res['Epochs']=epochs
    cumiliated_res['Optimizer']=Optimizer
    cumiliated_res['Activation_Function']=Activation
    cumiliated_res['Train_Loss']=history.history['loss']
    cumiliated_res['Train_Accuracy']=history.history['accuracy']
    cumiliated_res['Train_Precision']=history.history['precision']
    cumiliated_res['Train_Recall']=history.history['recall']
    cumiliated_res['Val_Loss']=history.history['val_loss']
    cumiliated_res['Val_Accuracy']=history.history['val_accuracy']
    cumiliated_res['Val_Precision']=history.history['val_precision']
    cumiliated_res['Val_Recall']=history.history['val_recall']
    convertDictionary = pd.DataFrame(cumiliated_res)
    Training_Output_Results = Training_Output_Results.append(convertDictionary)
    return Training_Output_Results

In [ ]:
Optimisers = ['RMSprop','Adam','Adadelta','Adagrad']
Activation_function =['relu','sigmoid','softmax','tanh','softsign','selu','elu']

# Optimizer Analytics

In [ ]:
Training_Output_Results =pd.DataFrame(columns=['Epochs','Optimizer','Activation_Function','Train_Loss','Train_Accuracy','Train_Precision','Train_Recall',                                             'Val_Loss','Val_Accuracy','Val_Precision','Val_Recall'])
def Optimise_verify(Training_Output_Results):
    for opt in Optimisers:
        model = build_model(Activation_function[0], final_class , w , h)
        METRICS = [
                'accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')
        ]  
        model.compile(
                optimizer=opt,
                loss='categorical_crossentropy',
                metrics=METRICS
            )
        history = model.fit(X_train, y_train, epochs=10, validation_split=0.3, batch_size=15,verbose=0,shuffle=True)
        Training_Output_Results = wrap(Training_Output_Results , opt,Activation_function[0],history)
        print('---------------------Round for '+opt+' Completed-----------------------------------------')
    return Training_Output_Results
    
    
Training_Output_Results = Optimise_verify(Training_Output_Results)

In [ ]:
Training_Output_Results=Training_Output_Results.sample(frac = 1) 
print(Training_Output_Results.shape)
Training_Output_Results.to_csv('Optimizer_64*64_data.csv', index = False) 
Training_Output_Results.head()

# Activation Analytics

In [ ]:
Training_Output_Results =pd.DataFrame(columns=['Epochs','Optimizer','Activation_Function','Train_Loss','Train_Accuracy','Train_Precision','Train_Recall',
                                              'Val_Loss','Val_Accuracy','Val_Precision','Val_Recall'])
def Activation_verify(Training_Output_Results):
    for act in Activation_function:
        model = build_model(act,final_class,w,h)
        METRICS = [
                'accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')
        ]  
        model.compile(
                optimizer=Optimisers[0],
                loss='categorical_crossentropy',
                metrics=METRICS
            )
        history = model.fit(X_train, y_train, epochs=10, validation_split=0.3, batch_size=15,verbose=0,shuffle=True)
        Training_Output_Results = wrap(Training_Output_Results , Optimisers[0],act,history)
        print('---------------------Round for '+act+' Completed-----------------------------------------')
    return Training_Output_Results
    
    
Training_Output_Results = Activation_verify(Training_Output_Results)

In [ ]:
Training_Output_Results=Training_Output_Results.sample(frac = 1) 
print(Training_Output_Results.shape)

Training_Output_Results.to_csv('Activation_64*64_data.csv', index = False)
Training_Output_Results.head()

# Ploting

In [ ]:
opt = pd.read_csv('./Optimizer_64*64_data.csv')
act = pd.read_csv('./Activation_64*64_data.csv')

# Optimization

In [ ]:
import plotly.express as px
import plotly.io as pio
import statsmodels.api as sm
scatterplot = px.area(
    data_frame=opt,
    x="Epochs",
    y="Train_Accuracy",
    color="Optimizer",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","black"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Train Accuracy',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)
pyo.iplot(scatterplot, filename = 'Opt_train_acc')

Adam and RMSprop shows best accuracies namely 93.2 and 98.5 %

In [ ]:
scatterplot = px.area(
    data_frame=opt,
    x="Epochs",
    y="Train_Loss",
    color="Optimizer",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Train Loss',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Opt_train_loss')

RMSprop 0.064
Adamgrad 0.0634
again a close call to distinguish

In [ ]:
scatterplot = px.area(
    data_frame=opt,
    x="Epochs",
    y="Train_Precision",
    color="Optimizer",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Train Precision',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Opt_train_prec')

Adam 99.9
RMSprop 99.6
Again a close boundary

In [ ]:
scatterplot = px.area(
    data_frame=opt,
    x="Epochs",
    y="Train_Recall",
    color="Optimizer",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Train Recall',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Opt_train_recall')

Both adam and Rmsprop miserably fail the recall test and Adagrad does best with 88.8%

In [ ]:
scatterplot = px.area(
    data_frame=opt,
    x="Epochs",
    y="Val_Accuracy",
    color="Optimizer",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Validation Accuracy',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Opt_val_acc')

In validation RMS 99.4 and Adam 66.4 this creates a good margin to distinguish both

In [ ]:
scatterplot = px.area(
    data_frame=opt,
    x="Epochs",
    y="Val_Precision",
    color="Optimizer",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Validation Precision',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Opt_val_prec')

In [ ]:
scatterplot = px.area(
    data_frame=opt,
    x="Epochs",
    y="Val_Recall",
    color="Optimizer",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Validation Recall',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Opt_val_recall')

In [ ]:
scatterplot = px.area(
    data_frame=opt,
    x="Epochs",
    y="Val_Loss",
    color="Optimizer",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Validation Loss',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Opt_val_loss')

# Activation Function

In [ ]:
import plotly.express as px
import plotly.io as pio
import statsmodels.api as sm
scatterplot = px.area(
    data_frame=act,
    x="Epochs",
    y="Train_Accuracy",
    color="Activation_Function",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Train Accuracy',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Act_train_acc')

In [ ]:
import plotly.express as px
import plotly.io as pio
import statsmodels.api as sm
scatterplot = px.area(
    data_frame=act,
    x="Epochs",
    y="Train_Loss",
    color="Activation_Function",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Train Loss',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Act_train_loss')

In [ ]:
import plotly.express as px
import plotly.io as pio
import statsmodels.api as sm
scatterplot = px.area(
    data_frame=act,
    x="Epochs",
    y="Train_Precision",
    color="Activation_Function",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Train Precision',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Act_train_prec')

In [ ]:
import plotly.express as px
import plotly.io as pio
import statsmodels.api as sm
scatterplot = px.area(
    data_frame=act,
    x="Epochs",
    y="Train_Recall",
    color="Activation_Function",                                              # set opacity of markers
    color_discrete_sequence=["red","orange","blue","green"],   # set marker colors. When color colum isn't numeric data
   
    #facet_col='Optimizer',       # assign marks to subplots in the horizontal direction
    #facet_col_wrap=2,           # maximum number of subplot columns. Do not set facet_row
    
    #log_x=True,                 # x-axis is log-scaled
    #log_y=True,                 # y-axis is log-scaled
    
    title='Train Recall',           # figure title
    #width=500,                  # figure width in pixels
    #height=500,                # igure height in pixels
    template='presentation',     # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
)

# print(scatterplot)

pyo.iplot(scatterplot, filename = 'Act_train_rec')

In [ ]:
import plotly.graph_objects as go
tab_opt = opt[opt['Epochs']==10]
final_col = np.delete(tab_opt.columns[0:], [0,2])
fig = go.Figure(data=[go.Table(
    header=dict(values=list(final_col),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tab_opt.Optimizer , tab_opt.Train_Loss,tab_opt.Train_Accuracy,tab_opt.Train_Precision,tab_opt.Train_Recall,tab_opt.Val_Loss,tab_opt.Val_Accuracy,tab_opt.Val_Precision,tab_opt.Val_Recall],
               fill_color='lavender',
               align='left'))
])

fig.show()

In [ ]:
import plotly.graph_objects as go
ty =opt[opt['Epochs']==10].iloc[:,3:]
nm = ty.columns
ty = ty.values.tolist()
data = []

for j in range(len(nm)):
        lt = []
        for i in range(len(Optimisers)):
            lt.append(ty[i][j])
            
        data.append(go.Bar(name = nm[j],x=Optimisers, y=lt))
fig = go.Figure(data=data)
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [ ]:
import plotly.graph_objects as go
tab_opt = act[act['Epochs']==10]
final_col = np.delete(tab_opt.columns[0:], [0,1])
fig = go.Figure(data=[go.Table(
    header=dict(values=list(final_col),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tab_opt.Activation_Function , tab_opt.Train_Loss,tab_opt.Train_Accuracy,tab_opt.Train_Precision,tab_opt.Train_Recall,tab_opt.Val_Loss,tab_opt.Val_Accuracy,tab_opt.Val_Precision,tab_opt.Val_Recall],
               fill_color='lavender',
               align='left'))
])

fig.show()

In [ ]:
import plotly.graph_objects as go
ty =act[act['Epochs']==10].iloc[:,3:]
nm = ty.columns
ty = ty.values.tolist()
data = []

for j in range(len(nm)):
        lt = []
        for i in range(len(Activation_function)):
            lt.append(ty[i][j])
            
        data.append(go.Bar(name = nm[j],x=Activation_function, y=lt))
fig = go.Figure(data=data)
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

Optimizer : RMSprop
Activation : Relu

# Final Model

In [ ]:
def Plot(history , name , model):
    model.save(name+'.h5')
    epochs = range(1,len(history.history['loss']) + 1)
    epochs = list(epochs)
    fig = make_subplots(rows=2, cols=4,subplot_titles=("Train Loss", "Train Accuracy" , "Train Precision","Train Recall", "Validation Loss", "Validation Accuracy",
                                                      "Validation Precision","Validation Recall"))
    fig.add_trace(go.Scatter(x=epochs, y=history.history['loss']), row=1, col=1)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['accuracy']), row=1, col=2)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['precision']), row=1, col=3)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['recall']), row=1, col=4)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['val_loss']), row=2, col=1)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['val_accuracy']), row=2, col=2)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['val_precision']), row=2, col=3)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['val_recall']), row=2, col=4)
    fig.update_layout(showlegend=False,height=1000, width=1200, title_text=name)
    pyo.iplot(fig, filename = 'Act_train_rec')

object : the model to train.      
-> X : our training data. Can be Vector, array or matrix      
-> Y : our training labels. Can be Vector, array or matrix       
-> Batch_size : it can take any integer value or NULL and by default, it will
be set to 32. It specifies no. of samples per gradient.      
-> Epochs : an integer and number of epochs we want to train our model for.      
-> Verbose : specifies verbosity mode(0 = silent, 1= progress bar, 2 = one
line per epoch).      
-> Shuffle : whether we want to shuffle our training data before each epoch.      
-> steps_per_epoch : it specifies the total number of steps taken before
one epoch has finished and started the next epoch. By default it values is set to NULL.

In [ ]:
model = build_model('relu', final_class ,w , h)
METRICS = [
                'accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')
]  
model.compile(
                optimizer='RMSprop',
                loss='categorical_crossentropy',
                metrics=METRICS
        )
history = model.fit(X_train, y_train, epochs=50, validation_split=0.3, batch_size=15,verbose=1,shuffle=True)

In [ ]:
Plot(history , 'final_model',model)

In [ ]:
import visualkeras
visualkeras.layered_view(model)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png',show_shapes=True)

In [ ]:
from matplotlib import pyplot
from matplotlib.pyplot import figure
figure(num=None, figsize=(25, 30), dpi=80, facecolor='w', edgecolor='k')
filters, biases = model.layers[1].get_weights()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
# plot first few filters
n_filters, ix = 6, 1
for i in range(n_filters):
	# get the filter
	f = filters[:, :, :, i]
	# plot each channel separately
	for j in range(3):
		# specify subplot and turn of axis
		ax = pyplot.subplot(n_filters,3 , ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		pyplot.imshow(f[:, :, j])
		ix += 1
# show the figure
pyplot.show()

In [ ]:
from matplotlib import pyplot
from matplotlib.pyplot import figure
figure(num=None, figsize=(25, 30), dpi=80, facecolor='w', edgecolor='k')
filters, biases = model.layers[1].get_weights()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
# plot first few filters
n_filters, ix = 6, 1
for i in range(n_filters):
	# get the filter
	f = filters[:, :, :, i]
	# plot each channel separately
	for j in range(3):
		# specify subplot and turn of axis
		ax = pyplot.subplot(n_filters,3 , ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		pyplot.imshow(f[:, :, j],cmap='gray')
		ix += 1
# show the figure
pyplot.show()

In [ ]:
plt.figure(figsize=(25,8))
plt.imshow(X[66].reshape(w,h))
plt.title(enc.inverse_transform(y[0].reshape(1,10))[0][0],size = 20)
plt.show()

In [ ]:
from keras.models import Model
from matplotlib.pyplot import figure
from numpy import expand_dims
def image_transform_gray(image):
    img = expand_dims(image, axis=0)
    model1 = Model(inputs=model.inputs, outputs=model.layers[0].output)
    feature_maps = model1.predict(img)
    figure(num=None, figsize=(25, 30), dpi=80, facecolor='w', edgecolor='k')
    square = 4
    ix = 1
    for _ in range(square):
        for _ in range(square):
            # specify subplot and turn of axis
            ax = pyplot.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            # plot filter channel in grayscale
            pyplot.imshow(feature_maps[0, :, :, ix-1],cmap='gray')
            ix += 1
    # show the figure
    pyplot.show()
def image_transform(image):
    img = expand_dims(image, axis=0)
    model1 = Model(inputs=model.inputs, outputs=model.layers[0].output)
    feature_maps = model1.predict(img)
    figure(num=None, figsize=(25, 30), dpi=80, facecolor='w', edgecolor='k')
    square = 4
    ix = 1
    for _ in range(square):
        for _ in range(square):
            # specify subplot and turn of axis
            ax = pyplot.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            # plot filter channel in grayscale
            pyplot.imshow(feature_maps[0, :, :, ix-1])
            ix += 1
    # show the figure
    pyplot.show()

In [ ]:
image_transform(X[66])
image_transform_gray(X[66])

In [ ]:
plt.figure(figsize=(25,8))
plt.imshow(X[56].reshape(w,h))
plt.title(enc.inverse_transform(y[0].reshape(1,10))[0][0],size = 20)
plt.show()

In [ ]:
image_transform(X[56])
image_transform_gray(X[56])

In [ ]:
plt.figure(figsize=(25,8))
plt.imshow(X[1566].reshape(w,h))
plt.title(enc.inverse_transform(y[0].reshape(1,10))[0][0],size = 20)
plt.show()

In [ ]:
image_transform(X[1566])
image_transform_gray(X[1566])

# Test The Results

In [ ]:
y_pred = model.evaluate(X_test , y_test,verbose =1)

In [ ]:
y_pred[1]

In [ ]:
history.history['loss'][49]
import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Bar(name = 'Accuracy',x=['Training','Validation','Real World Data'], y=[history.history['accuracy'][49] ,history.history['val_accuracy'][49],y_pred[1] ]),
    go.Bar(name = 'Precision',x=['Training','Validation','Real World Data'], y=[history.history['precision'][49] ,history.history['val_precision'][49],y_pred[2] ]),
    go.Bar(name = 'Loss',x=['Training','Validation','Real World Data'], y=[history.history['loss'][49] ,history.history['val_loss'][49],y_pred[0] ]),

])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_yaxes(type = "log")
pyo.iplot(fig, filename = 'Act_train_rec')

In [ ]:
model.predict(X_test)

In [ ]:
y_prediction = model.predict(X_test)
def binary_classify(y_pred):
    for inp in y_pred:
        maximum = 0
        index = 0
        for i in range(10):
            if(maximum != max(maximum,inp[i])):
                maximum = max(maximum,inp[i])
                index = i
            inp[i] = 0
        inp[index]=1
    return y_pred
y_prediction  = binary_classify(y_prediction)

In [ ]:
def create_result(y):
    y_final = []
    for i in range(y.shape[0]):
        y_final.append(enc.inverse_transform(y[i].reshape(1,10))[0][0])
    return y_final 
def remove_none(y , y_pred):
    index = []
    for i in range(len(y)-1,0,-1):
        if y_pred[i] == None :
            del y[i]
            del y_pred[i]
        
    return y , y_pred
def label_encode(y , y_pred):
    le = preprocessing.LabelEncoder()
    le.fit(y_pred)
    print(le.classes_)
    y = le.transform(y)
    y_pred = le.transform(y_pred)
    return y , y_pred

y_class_result = create_result(y_prediction)
y_class_desired = create_result(y_test)

In [ ]:
y_label_desired , y_label_result = label_encode(y_class_desired , y_class_result) 

In [ ]:
from sklearn.metrics import classification_report
tn = []
for cat in enc.categories_[0].reshape(10,1):
    tn.append(cat[0])
target_names = tn
print(classification_report(y_label_desired, y_label_result, target_names=target_names))

In [ ]:
count = 1
f = plt.figure(figsize=(20,24))
for i in range(20):
    ind = random.sample(list(y_label_result),1)[0]
    img = X_test[ind]
    Class = str(y_class_desired[ind]) + '  vs  '+str(y_class_result[ind])
    ax = f.add_subplot(5, 4,count)
    ax = plt.imshow(img.reshape(w,h))
    ax = plt.title(Class,fontsize= 11)
    count = count + 1
plt.suptitle("Hand Sign Images", size = 32)
plt.show()